# First configurations

In [2]:
import glob
import tensorflow as tf
import numpy as np

In [3]:
FEATURES_PATH = "/home/admin/FEUP-Synthesizing-Audio-from-Textual-Input/audioset/data/features/audioset_v1_embeddings/bal_train"
BATCH_SIZE = 32

# Load the AudioSet Dataset

In [4]:
# Get a list of all TFRecord files in a directory
tfrecord_files = glob.glob(f"{FEATURES_PATH}/*.tfrecord")

# Load all TFRecord files into a single dataset
dataset = tf.data.TFRecordDataset(tfrecord_files).batch(BATCH_SIZE)

2023-07-28 12:37:18.606550: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


# Pass to pytorch

In [18]:
import tensorflow as tf

# Constants
FEATURES_PATH = "/home/admin/FEUP-Synthesizing-Audio-from-Textual-Input/audioset/data/features/audioset_v1_embeddings/bal_train"

# Function to read and print feature keys from TFRecord files
def inspect_feature_keys(tfrecord_file):
    for record in tf.data.TFRecordDataset(tfrecord_file):
        example = tf.train.Example()
        example.ParseFromString(record.numpy())
        
        # Print the feature keys for each example
        print(list(example.features.feature.keys()))

# Get a list of all TFRecord files in the directory
tfrecord_files = tf.io.gfile.glob(f"{FEATURES_PATH}/*.tfrecord")

# Loop through each TFRecord file and inspect the feature keys
for file in tfrecord_files:
    print(f"Inspecting file: {file}")
    inspect_feature_keys(file)
    print()


Inspecting file: /home/admin/FEUP-Synthesizing-Audio-from-Textual-Input/audioset/data/features/audioset_v1_embeddings/bal_train/YH.tfrecord
['end_time_seconds', 'video_id', 'start_time_seconds', 'labels']
['end_time_seconds', 'video_id', 'start_time_seconds', 'labels']
['end_time_seconds', 'video_id', 'start_time_seconds', 'labels']
['end_time_seconds', 'video_id', 'start_time_seconds', 'labels']

Inspecting file: /home/admin/FEUP-Synthesizing-Audio-from-Textual-Input/audioset/data/features/audioset_v1_embeddings/bal_train/Oi.tfrecord
['end_time_seconds', 'video_id', 'start_time_seconds', 'labels']
['end_time_seconds', 'video_id', 'start_time_seconds', 'labels']
['labels', 'video_id', 'start_time_seconds', 'end_time_seconds']
['end_time_seconds', 'video_id', 'start_time_seconds', 'labels']
['end_time_seconds', 'video_id', 'start_time_seconds', 'labels']
['end_time_seconds', 'video_id', 'start_time_seconds', 'labels']

Inspecting file: /home/admin/FEUP-Synthesizing-Audio-from-Textual-In

KeyboardInterrupt: 

## Convert to pytorch tensors

In [25]:
import tensorflow as tf
import torch
import h5py
import glob

# Constants
FEATURES_PATH = "/home/admin/FEUP-Synthesizing-Audio-from-Textual-Input/audioset/data/features/audioset_v1_embeddings/bal_train"
BATCH_SIZE = 32

# Get a list of all TFRecord files in a directory
tfrecord_files = glob.glob(f"{FEATURES_PATH}/*.tfrecord")

# Helper function to parse the TFRecord example and convert to TensorFlow tensors
def parse_example(example):
    feature_description = {
        # Replace these keys with the actual feature keys
        'labels': tf.io.FixedLenFeature([], tf.int64),
        'video_id': tf.io.FixedLenFeature([], tf.string),
        'start_time_seconds': tf.io.FixedLenFeature([], tf.float32),
        'end_time_seconds': tf.io.FixedLenFeature([], tf.float32),
    }
    example = tf.io.parse_single_example(example, feature_description)
    
    return (
        example['labels'],
        example['video_id'],
        example['start_time_seconds'],
        example['end_time_seconds']
        # Add more tensors if you have more features
    )

# Load all TFRecord files into a single dataset
dataset = tf.data.TFRecordDataset(tfrecord_files)
dataset = dataset.map(parse_example)
dataset = dataset.batch(BATCH_SIZE)

# Convert TensorFlow tensors to a list of PyTorch tensors
tensor_list = [(torch.tensor(labels.numpy(), dtype=torch.int64),
                video_id.numpy().decode('utf-8'),
                torch.tensor(start_time.numpy(), dtype=torch.float32),
                torch.tensor(end_time.numpy(), dtype=torch.float32)) for labels, video_id, start_time, end_time in dataset]


2023-07-28 13:00:24.256808: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: labels.  Can't parse serialized Example.
2023-07-28 13:00:24.256889: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: labels.  Can't parse serialized Example.
2023-07-28 13:00:24.256966: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: labels.  Can't parse serialized Example.
2023-07-28 13:00:24.257040: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: labels.  Can't parse serialized Example.
2023-07-28 13:00:24.257978: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Key: labels.  Can't parse serialized Example.
2023-07-28 13:00:24.258018: W tensorflow/core/framework

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_4_device_/job:localhost/replica:0/task:0/device:CPU:0}} Key: labels.  Can't parse serialized Example.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]] [Op:IteratorGetNext] name: 

In [26]:
# After creating the dataset, add debugging mode
dataset = dataset.apply(tf.data.experimental.enable_debug_mode())

# Then continue with the rest of the code
dataset = dataset.batch(BATCH_SIZE)
tensor_list = [(torch.tensor(labels.numpy(), dtype=torch.int64),
                video_id.numpy().decode('utf-8'),
                torch.tensor(start_time.numpy(), dtype=torch.float32),
                torch.tensor(end_time.numpy(), dtype=torch.float32)) for labels, video_id, start_time, end_time in dataset]


TypeError: 'NoneType' object is not callable